In [ ]:
from collections import Counter, defaultdict
from contextlib import suppress
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
from ruamel.yaml import YAML
from ruamel.yaml.scanner import ScannerError
from tqdm.auto import tqdm

In [ ]:
months = [
    "2022-06",
    "2022-07",
    "2022-08",
    "2022-09",
    "2022-10",
    "2022-11",
    "2022-12",
    "2023-01",
    "2023-02",
    "2023-03",
    "2023-04",
    "2023-05",
    "2023-06",
    "2023-07",
    "2023-08",
    "2023-09",
    "2023-10",
    "2023-11",
    "2023-12",
    "2024-01",
    "2024-02",
    "2024-03",
    "2024-04",
    "2024-05",
    "2024-06",
    "2024-07",
]
pre_commit_root = Path("pre-commits")
repo_dir_list = list(pre_commit_root.iterdir())
yaml = YAML(typ="safe")
yaml.allow_duplicate_keys = True

## Explanation

The following fairly dense logic iterates through each repos directory and each months .pre-commit-config.yaml files. Not all repos have a commit each calendar month so in that case if there is a preceding month that does have data about pre-commit tool the previous month data just fills forward into the missing month and that chaining repeats until a month without missing data is found.

Some repos didn't have .pre-commit-config.yaml files for several months at the beginning of the requested timespan so they have no tools to count for those initial missing month periods (which is why I skip a few months of collected data when plotting the output). Ruff wasn't published until late Aug 2022 - I think the Ruff pre-commit tool didn't get published until a couple of months or so after that.

Also note there's quite a few hacky looking conditional rules in the center of these loops to try and identify tool names. pre-commit can run any sort of tool - the easy case is the Github repo links, but there's also a lot of instances in repos of the pre-commit running local scripts and python modules so the conditional rules try to handle the majority of the cases I observed.

In [ ]:
month_repo_counters = defaultdict(Counter)
for repo_dir in tqdm(repo_dir_list):
    tool_names = []
    precommit_dir_list = [(repo_dir / month) for month in months]
    for precommit_dir in precommit_dir_list:
        tool_names = []
        if (precommit_dir / ".pre-commit-config.yaml").is_file():
            tool_names.append("precommit_count")
            with suppress(ScannerError):
                yaml_data = yaml.load(precommit_dir / ".pre-commit-config.yaml")
                repos = yaml_data.get("repos", yaml_data) if isinstance(yaml_data, dict) else yaml_data
                tools_per_precommit_file = set()
                for repo in repos:
                    if "http" in repo["repo"]:
                        tool_name = repo["repo"].split("/")[-1] if repo["repo"][-1] != "/" else repo["repo"].split("/")[-2]

                        if tool_name.find("mirrors-") == 0:
                            tool_name = tool_name[len("mirrors-") :]

                        if tool_name.endswith(".git"):
                            tool_name = tool_name[:-4]

                        if tool_name == "black-pre-commit-mirror":
                            tool_name = "black"
                        elif tool_name == "ruff-pre-commit":
                            tool_name = "ruff"
                        if tool_name not in tools_per_precommit_file:
                            tools_per_precommit_file.add(tool_name)
                            tool_names.append(tool_name)
                    elif len(repo["hooks"]) > 0:
                        hook = repo["hooks"][0]
                        if "entry" not in hook:
                            continue
                        tool_name = hook["entry"]
                        run_pos = tool_name.find(" run ")
                        if run_pos >= 0:
                            tool_name = tool_name[run_pos + len(" run ") :].split(" ")[0]
                        if tool_name.find("python -m ") == 0:
                            tool_name = tool_name[len("python -m ") :].split(" ")[0]
                        elif tool_name.find("python ") == 0:
                            tool_name = tool_name[len("python ") :].split(" ")[0]
                        elif tool_name.find("python3 ") == 0:
                            tool_name = tool_name[len("python3 ") :].split(" ")[0]
                        elif tool_name.find("./activated.py python ") == 0:
                            tool_name = tool_name[len("./activated.py python ") :].split(" ")[0]
                        elif tool_name.find("./activated.py ") == 0:
                            tool_name = tool_name[len("./activated.py ") :].split(" ")[0]
                        elif tool_name == "poetry" and "args" in hook and len(hook["args"]) >= 2 and hook["args"][0] == "run":
                            tool_name = hook["args"][1]

                        if tool_name.endswith(".git"):
                            tool_name = tool_name[:-4]

                        if tool_name.find("mirrors-") == 0:
                            tool_name = tool_name[len("mirrors-") :]

                        tool_name = tool_name.split(" ")[0]

                        if tool_name == "black-pre-commit-mirror":
                            tool_name = "black"
                        elif tool_name == "ruff-pre-commit":
                            tool_name = "ruff"

                        if tool_name not in tools_per_precommit_file:
                            tools_per_precommit_file.add(tool_name)
                            tool_names.append(tool_name)
        month_repo_counters[precommit_dir.name].update(tool_names)

In [ ]:
ordered_counters = [month_repo_counters[month] for month in months]
df = pd.DataFrame(data=ordered_counters, index=months)
df.index = pd.to_datetime(df.index)
precommit_count = df.pop("precommit_count")

scaled_df = df.div(precommit_count, axis=0) * 100.0

In [ ]:
scaled_df.iloc[-1:, :].sum().sort_values(ascending=False).iloc[:20].to_dict()

In [ ]:
columns = ["black", "flake8", "isort", "pyupgrade", "mypy", "ruff"]

In [ ]:
def percent_formatter(x, pos):
    return f"{int(x)}%"

In [ ]:
ax = scaled_df.loc[:, columns].plot(figsize=(10.76, 7.68), title="Python Github Repo Pre-commit Tools")
ax.set_xlabel("Date")
ax.set_ylabel("Repo Percentage")
ax.grid(True, which="both", alpha=0.1)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(percent_formatter))

In [ ]:
ax.get_figure().savefig("ruff_growth.png", bbox_inches="tight", pad_inches=0.1)

In [ ]:
with plt.style.context("grayscale"):
    # fig, ax = plt.subplots(figsize=(10.76, 7.68))
    fig = plt.figure(figsize=(10.76, 7.68))
    ax = fig.add_axes([0.1, 0.1, 0.9, 0.9])
    for column in columns:
        if column == "ruff":
            ax.plot(scaled_df.index, scaled_df[column], label=column, color="fuchsia", linewidth=5)
        else:
            ax.plot(scaled_df.index, scaled_df[column], label=column)
    ax.set_xlabel("Date")
    ax.set_ylabel("Repo Percentage")
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(percent_formatter))
    # You may need to manually install the Alliance Platt or Palace Script MT fonts manually to reproduce
    # this plot
    ax.text(
        0.42, 1.05, "Ruff", transform=ax.transAxes, ha="center", va="bottom", weight="bold", fontsize=36, color="fuchsia", family="Alliance"
    )
    ax.text(0.48, 1.05, "is\nthe\nnew", transform=ax.transAxes, ha="left", va="bottom", fontsize=12, weight="normal")
    ax.text(
        0.59,
        1.03,
        "Black",
        transform=ax.transAxes,
        ha="center",
        va="bottom",
        fontsize=64,
        weight="bold",
        color="black",
        family="Palace Script MT",
    )
    ax.text(
        0.5, -0.1, "Pre-commit tool usage over time in top 1000 Python repos by GitHub stars", transform=ax.transAxes, ha="center", va="top"
    )
    ax.legend()
    ax.grid(True, alpha=0.1)

In [ ]:
fig.savefig("ruff_is_the_new_black.png", bbox_inches="tight", pad_inches=0.1)

In [ ]:
df.fillna(0.0).to_csv("repo_count_per_precommit_tool.csv")

In [ ]:
repo_list = [str(path.stem) for path in repo_dir_list]

file_path = Path("repos.txt")

# Open the file in write mode and write each element to the file
with file_path.open("w") as file:
    for repo in repo_list:
        file.write(f"{repo}\n")